In [1]:
using Statistics
using LinearAlgebra
using Random
using Plots
using LaTeXStrings
using TensorCrossInterpolation
import TensorCrossInterpolation as TCI
# using qtt_option
# import qtt_option as qo

In [2]:
countelem(M) = sum([prod(size(x)) for x in M])

countelem (generic function with 1 method)

In [3]:
function tci_oneshot(func, d, localdims, firstpivot, tol)
    BLAS.set_num_threads(4)
    #func_q = GenNDfunc(d, func)

    for isearch in 1:100
        p = TCI.optfirstpivot(func, localdims, firstpivot) # search optimal fist pivot
        if abs(func(p)) > abs(func(firstpivot))
            firstpivot = p
        end
    end

    # execute tci2
    qtt, ranks, errors = TCI.crossinterpolate2(
        ComplexF64,
        func, 
        localdims, 
        [firstpivot], 
        tolerance = tol, 
        maxiter = 10, 
        verbosity = 1, 
        loginterval = 1,
        pivotsearch = :rook,
        # normalizeerror = false,
    )

    return qtt, errors
end    

tci_oneshot (generic function with 1 method)

In [4]:
function def_nonDiag(d, mtr::Matrix{Float64}, nondiag::Float64)
    for i in 1:d, j in 1:d
        if i != j
            mtr[i, j] = nondiag
        end
    end
    return mtr
end

def_nonDiag (generic function with 1 method)

$$
v(\vec{S}(T)) = \max \Bigl\lbrace \min \lbrace S^1_T, \ldots, S^d_T \rbrace - K, 0 \Bigl\rbrace
$$

In [5]:
function payoff_v(x::Vector{Float64}, K::Float64)
    return maximum([(minimum(exp.(x)) - K), 0.0])
end
;

$$
q(\vec{x}|\vec{x}_0) = \frac{1}{\sqrt{(2 \pi)^d \det \Sigma}} \exp\left(-\frac{1}{2}\left(\vec{x}-\vec{\mu}\right)^T \Sigma^{-1} \left(\vec{x}-\vec{\mu}\right)\right)
$$

$$
\mu_j = x_0^j+r_j T-\frac{1}{2} \sigma_j^2 \Sigma_{j j} T 
$$

In [6]:
function normal_dstrb_q(d::Int, x::Vector{Float64}, x0::Vector{Float64}, σ::Vector{Float64}, Σ::Matrix{Float64}, T::Float64, r::Float64)
    #μ = [x0[i] + r * T - σ[i]^2 * Σ[i,i] * T / 2 for i in 1:d]
    μ = [x0[i] + r * T - Σ[i,i] * T / 2 for i in 1:d]
    mtrx = transpose(x .- μ) * inv(Σ) * (x .- μ)
    
    return exp(-mtrx/2)/sqrt((2*π)^d * det(Σ))
end

normal_dstrb_q (generic function with 1 method)

In [7]:
function func(idxs::Vector{Int}, f::Function, N::Int, cut::Float64, slice::Float64)
    return f( cut .* ( idxs ) .+ slice )
end

func (generic function with 1 method)

In [8]:
function evaluate_options_(random_combinations, tt_option)
    result_hako = []
    result_time = []
    for i in random_combinations
        #@show i
        time_inner = @elapsed begin
            result = qo._evaluate(tt_option, i)
        end
        push!(result_hako, result)
        push!(result_time, time_inner)
    end
    return result_hako, result_time
end

evaluate_options_ (generic function with 1 method)

In [9]:
d = 3
T = 1.0
r = 0.01
K = 100.0
s0 = fill(100.0, d)
x0 = log.(s0)
σ = fill(0.2, d)

nondiag = 1/3
Σ_ = def_nonDiag(d, Matrix{Float64}(I, d, d), nondiag)
σ_ = Matrix{Float64}(I, d, d) .* σ[1]
Σ = σ_ * Σ_ * σ_

3×3 Matrix{Float64}:
 0.04       0.0133333  0.0133333
 0.0133333  0.04       0.0133333
 0.0133333  0.0133333  0.04

In [10]:
v(x) = payoff_v(x, K)
q(x) = normal_dstrb_q(d, x, x0, σ, Σ, T, r)

q (generic function with 1 method)

In [11]:
N = 200
cut = 0.05
slice = 0.0

@show slice
@show N*cut + slice

idxs_ = fill(80, d) #fill(Int(N/2), d)
xs = cut .* ( idxs_ ) .+ slice
xsp =  cut .* ( idxs_ .+ 2 ) .+ slice
xsm =  cut .* ( idxs_ .- 2 ) .+ slice

v_pre(idx) = func(idx, v, N, cut, slice)
q_pre(idx) = func(idx, q, N, cut, slice)

slice = 0.0
N * cut + slice = 10.0


q_pre (generic function with 1 method)

In [12]:
@show v(xs) - v_pre(idxs_)
@show q(xs) - q_pre(idxs_)
@show v(xsp) - v_pre(idxs_ .+ 2)
@show q(xsp) - q_pre(idxs_ .+ 2)
@show v(xsm) - v_pre(idxs_ .- 2)
@show q(xsm) - q_pre(idxs_ .- 2)
;

v(xs) - v_pre(idxs_) = 0.0
q(xs) - q_pre(idxs_) = 0.0
v(xsp) - v_pre(idxs_ .+ 2) = 0.0
q(xsp) - q_pre(idxs_ .+ 2) = 0.0
v(xsm) - v_pre(idxs_ .- 2) = 0.0
q(xsm) - q_pre(idxs_ .- 2) = 0.0


In [13]:
localdims = fill(N, d)
firstpivot = rand(1:N, d)

v_tci = TCI.ThreadedBatchEvaluator{Float64}(v_pre, localdims)
q_tci = TCI.ThreadedBatchEvaluator{Float64}(q_pre, localdims)

(::TensorCrossInterpolation.ThreadedBatchEvaluator{Float64}) (generic function with 3 methods)

In [14]:
tol_v = 1e-6
tol_q = 1e-13

1.0e-13

In [15]:
tci_time = @elapsed begin
    @time qtt_v, errors_v = tci_oneshot(v_tci, d, localdims, firstpivot, tol_v)
    @time qtt_q, errors_q = tci_oneshot(q_tci, d, localdims, firstpivot, tol_q)
end

iteration = 1, rank = 8, error= 0.0, maxsamplevalue= 2000.6455894201777, nglobalpivot=2
  Rejected 2 global pivots added in the previous iteration, errors are [0.0, 1.4210854715202004e-14]
iteration = 2, rank = 15, error= 0.0, maxsamplevalue= 11988.380730216988, nglobalpivot=0
iteration = 3, rank = 26, error= 0.0, maxsamplevalue= 11988.380730216988, nglobalpivot=0
iteration = 4, rank = 50, error= 0.0, maxsamplevalue= 11988.380730216988, nglobalpivot=1
  Rejected 1 global pivots added in the previous iteration, errors are [0.0]
iteration = 5, rank = 77, error= 0.0, maxsamplevalue= 11988.380730216988, nglobalpivot=1
  Rejected 1 global pivots added in the previous iteration, errors are [2.7284841053187847e-12]
iteration = 6, rank = 91, error= 0.0, maxsamplevalue= 11988.380730216988, nglobalpivot=1
  Rejected 1 global pivots added in the previous iteration, errors are [0.0]
iteration = 7, rank = 95, error= 0.0, maxsamplevalue= 11988.380730216988, nglobalpivot=1
  Rejected 1 global pivots 

106.261013612

In [16]:
M1 = qtt_v.sitetensors
M2 = qtt_q.sitetensors

3-element Vector{Array{ComplexF64, 3}}:
 [2.381556103125557e-104 - 0.0im 1.0500318140296634e-101 - 0.0im … 2.0061723406391077e-153 - 0.0im 1.1920122109296986e-156 - 0.0im;;; 2.1803753700741906e-104 - 0.0im 9.61349323069494e-102 - 0.0im … 1.4144523103032066e-153 - 0.0im 8.404315085949312e-157 - 0.0im;;; -8.97766049390548e-104 + 0.0im -3.9581932112489566e-101 + 0.0im … -1.047793916225975e-152 + 0.0im -6.225665285858768e-156 + 0.0im;;; … ;;; -6.110276864571474e-104 + 0.0im -2.694189391043401e-101 + 0.0im … -2.6687584998629876e-153 + 0.0im -1.5857145228730737e-156 + 0.0im;;; 8.882612989938672e-104 - 0.0im 3.916264789392085e-101 - 0.0im … 1.1815522263918487e-152 - 0.0im 7.020394889470432e-156 - 0.0im;;; 1.1213173263513998e-103 + 0.0im 4.944933372717059e-101 + 0.0im … 2.1478927050155034e-153 + 0.0im 1.2762339595653431e-156 + 0.0im]
 [5.344300559037799e-115 - 0.0im 3.0553505890149696e-112 - 0.0im … 3.0479803566490317e-164 - 0.0im 1.4314701481180534e-167 - 0.0im; 1.545347139412884e-111 - 0.0im

In [17]:
smpl_indices_v = []
smpl_indices_q = []
for i in 1:100
    x = Int(trunc(N/100)) * i
    push!(smpl_indices_v, vcat(fill(Int(trunc(N*0.6)), d-1), x))
    push!(smpl_indices_q, vcat(fill(Int(trunc(N*0.65)), d-1), x))
end
@show smpl_indices_v
@show smpl_indices_q
;

smpl_indices_v = Any[[120, 120, 2], [120, 120, 4], [120, 120, 6], [120, 120, 8], [120, 120, 10], [120, 120, 12], [120, 120, 14], [120, 120, 16], [120, 120, 18], [120, 120, 20], [120, 120, 22], [120, 120, 24], [120, 120, 26], [120, 120, 28], [120, 120, 30], [120, 120, 32], [120, 120, 34], [120, 120, 36], [120, 120, 38], [120, 120, 40], [120, 120, 42], [120, 120, 44], [120, 120, 46], [120, 120, 48], [120, 120, 50], [120, 120, 52], [120, 120, 54], [120, 120, 56], [120, 120, 58], [120, 120, 60], [120, 120, 62], [120, 120, 64], [120, 120, 66], [120, 120, 68], [120, 120, 70], [120, 120, 72], [120, 120, 74], [120, 120, 76], [120, 120, 78], [120, 120, 80], [120, 120, 82], [120, 120, 84], [120, 120, 86], [120, 120, 88], [120, 120, 90], [120, 120, 92], [120, 120, 94], [120, 120, 96], [120, 120, 98], [120, 120, 100], [120, 120, 102], [120, 120, 104], [120, 120, 106], [120, 120, 108], [120, 120, 110], [120, 120, 112], [120, 120, 114], [120, 120, 116], [120, 120, 118], [120, 120, 120], [120, 120, 1

In [18]:
plot_v = plot(
    #collect(Int(trunc(N/100)):Int(trunc(N/100)):N),
    real.(v_pre.(smpl_indices_v))
)

plot!(
    plot_v,
    #collect(Int(trunc(N/100)):Int(trunc(N/100)):N),
    real.(evaluate_options_(smpl_indices_v, M1)[1])
)

UndefVarError: UndefVarError: `qo` not defined

In [19]:
plot(
    #collect(Int(trunc(N/100)):Int(trunc(N/100)):N),
    real.(v_pre.(smpl_indices_v)) .- real.(evaluate_options_(smpl_indices_v, M1)[1])
)

UndefVarError: UndefVarError: `qo` not defined

In [20]:
plot_q = plot(
    #yscale = :log10,
    #collect(20:20:2000),
    real.(q_pre.(smpl_indices_q))
)

plot!(
    plot_q,
    #collect(20:20:2000),
    real.(evaluate_options_(smpl_indices_q, M2)[1])
)

UndefVarError: UndefVarError: `qo` not defined

In [21]:
plot(
    collect(20:20:2000),
    real.(q_pre.(smpl_indices_q)) .-  real.(evaluate_options_(smpl_indices_q, M2)[1])
)

UndefVarError: UndefVarError: `qo` not defined

In [22]:
num_samples = 100
random_combinations = qo.generate_random_combinations(N, d, num_samples)
;

UndefVarError: UndefVarError: `qo` not defined

In [23]:
indices = []
for i in 1:N
    for j in 1:N
        push!(indices, [i, j])
    end
end

In [24]:
@show maximum(real.(v_pre.(indices)) .- real.(evaluate_options_(indices, M1)[1]))
@show maximum(real.(q_pre.(indices)) .- real.(evaluate_options_(indices, M2)[1]))

UndefVarError: UndefVarError: `qo` not defined

$$
V(\vec{p})=\mathbb{E}\left[e^{-rT}v(\vec{S}(T))\middle|\vec{S}_0\right]=e^{-rT} \int_{-\infty}^{\infty} v(\exp(\vec{x}))q(\vec{x}|\vec{x}_0)dx
$$

In [25]:
function create_AB(A::Array{ComplexF64,3}, B::Array{ComplexF64,3})
    # A_{i j k}
    # B_{l j m}
    dim_i, dim_j_A, dim_k = size(A)
    dim_l, dim_j_B, dim_m = size(B)
    @assert dim_j_A == dim_j_B "物理次元（j）が一致しません"

    # 縮約後のテンソルAB_{i l k m}を作成
    AB = zeros(ComplexF64, dim_i, dim_l, dim_k, dim_m)
    for j in 1:dim_j_A
        for i in 1:dim_i
            for l in 1:dim_l
                for k in 1:dim_k
                    for m in 1:dim_m
                        AB[i, l, k, m] += A[i, j, k] * B[l, j, m]
                    end
                end
            end
        end
    end

    # インデックスの結合： (i, l) -> x1, (k, m) -> x2
    #dim_x1 = dim_i * dim_l
    #dim_x2 = dim_k * dim_m
    #AB_matrix = reshape(AB, dim_x1, dim_x2)

    #return AB_matrix
    return AB
end


create_AB (generic function with 1 method)

In [26]:
function mps_inner_product_custom(M1::Vector{Array{ComplexF64,3}}, M2::Vector{Array{ComplexF64,3}})
    N = length(M1)
    @assert length(M2) == N "MPSの長さが一致しません"

    # 初期化
    M = nothing
    R = 0.0

    for i in 1:N
        A = M1[i]
        B = M2[i]
        AB = create_AB(A, B)

        dim_i, dim_l, dim_k, dim_m = size(AB)

        if M === nothing
            M = AB
        else
            min_dim_k = min(size(M, 3), size(AB, 1))
            min_dim_m = min(size(M, 4), size(AB, 2))

            for k in 1:min_dim_k
                for m in 1:min_dim_m
                    for o in 1:size(AB, 3)
                        for p in 1:size(AB, 4)
                            R += M[1, 1, k, m] * AB[k, m, o, p]
                        end
                    end
                end
            end
        end
    end

    @show R
    return R
end


mps_inner_product_custom (generic function with 1 method)

In [27]:
inner_product = mps_inner_product_custom(M1, M2)

R = 1423.5148787401486 + 0.0im


1423.5148787401486 + 0.0im

In [28]:
res = inner_product * exp(- r * T) * (cut^d)

0.17616883362951893 + 0.0im

In [29]:
ref = 3.344541197233318

3.344541197233318

In [30]:
real(ref - res)

3.1683723636037993

In [31]:
0.0003000939635962041
0.0012151027433864314

0.0012151027433864314

In [32]:
real(ref - res) / ref

0.9473264572805233